## ETK Filtering

In [ ]:
import Simulator
import Observation
import Statistics
import IEWParticleFilter

## Loading simulator 

For Kalman filtering we have to be able to propagate a Gaussian distribution in time and to have access to the linear operators

In [ ]:
timestamp = "2022_03_02-12_44_46"

grid, simulator = Simulator.from_file(timestamp)

## Definition of initial distribution 

The inital state is equipped with a Gaussian distribution specified by its mean and covariance. 

In [ ]:
statistics = Statistics.Statistics(simulator, N_e=50, safe_history=True)

In [ ]:
prior_args = Statistics.prior_args_from_file(timestamp)
statistics.set_prior(prior_args)

statistics.plot()

## Loading observations 

Observation positions which are fixed for a setup and values from the truth

In [ ]:
obs_timestamp = "2022_03_02-12_44_53"
observation = Observation.from_file(grid, timestamp, obs_timestamp)

## IEWPF filtering

Updating/conditioning the distributions at every observation time

In [ ]:
import numpy as np

ensemble_beforeDA = np.zeros((11,grid.N_x,statistics.ensemble.N_e))
ensemble_afterDA = np.zeros((11,grid.N_x,statistics.ensemble.N_e))

ensemble_beforeDA[0] = statistics.ensemble.ensemble
ensemble_afterDA[0] = statistics.ensemble.ensemble

In [ ]:
iewpFilter = IEWParticleFilter.IEWParticle(statistics, observation, beta=0.55)

for t in range(observation.N_obs):
    print(t)
    statistics.propagate(25, model_error=False)
    ensemble_beforeDA[t+1] = statistics.ensemble.ensemble
    iewpFilter.filter(statistics.ensemble.ensemble, observation.obses[t])
    ensemble_afterDA[t+1]  = statistics.ensemble.ensemble
    statistics.plot()

print(iewpFilter.betas)